In [1]:
#Checking if the sites working, this particular site is kinda weird

import requests
from bs4 import BeautifulSoup

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/115.0.0.0 Safari/537.36"
}

url = "https://sites.rootsweb.com/~wytttp/ghosttowns.htm"
response = requests.get(url, headers=headers)
response.encoding = "ISO-8859-1"
html = response.text

#Save HTML to a file you can inspect
with open("/tmp/debug_page.html", "w", encoding="utf-8") as f:
    f.write(html)
print("Saved page HTML to /tmp/debug_page.html")

#Parse and count h3 tags
soup = BeautifulSoup(html, "html.parser")
h3_tags = soup.find_all("h3")
print(f"Found {len(h3_tags)} <h3> tags on the page.")
for tag in h3_tags[:5]:
    print("Sample town name:", tag.get_text(strip=True))


Saved page HTML to /tmp/debug_page.html
Found 25 <h3> tags on the page.
Sample town name: Atlantic City
Sample town name: Bear River City
Sample town name: Benton
Sample town name: Bessemer
Sample town name: Bryan


In [2]:
import requests
from bs4 import BeautifulSoup
from simplekml import Kml
from IPython.display import FileLink
import re

def dms_to_decimal(deg, minute, sec, direction):
    decimal = float(deg) + float(minute) / 60 + float(sec) / 3600
    if direction in ['S', 'W']:
        decimal = -decimal
    return decimal

def parse_dms(dms_text):
    parts = re.findall(r'(\d+)°(\d+)\'([\d.]+)"?\s*([NSEW])', dms_text)
    if len(parts) == 2:
        (d1, m1, s1, dir1), (d2, m2, s2, dir2) = parts
        lat = dms_to_decimal(d1, m1, s1, dir1)
        lon = dms_to_decimal(d2, m2, s2, dir2)
        return lat, lon
    return None, None

url = "https://sites.rootsweb.com/~wytttp/ghosttowns.htm"
headers = {
    "User-Agent": "Mozilla/5.0"
}
response = requests.get(url, headers=headers)
response.encoding = "ISO-8859-1"
soup = BeautifulSoup(response.text, "html.parser")

towns = []

for h3 in soup.find_all("h3"):
    name = h3.get_text(strip=True)
    coord_tag = h3.find_next("a", href=True)
    if not coord_tag:
        continue
    coord_text = coord_tag.get_text(strip=True)
    lat, lon = parse_dms(coord_text)
    if lat is None or lon is None:
        continue

    status = "Unknown"
    next_text = coord_tag.find_next(string=True)
    while next_text and (not next_text.strip() or next_text.parent.name in ['a', 'h3']):
        next_text = next_text.find_next(string=True)
    if next_text:
        status = next_text.strip()

    towns.append((name, lat, lon, status))

print(f"Found {len(towns)} ghost towns.")

#Create KML file
kml = Kml()
for name, lat, lon, status in towns:
    pnt = kml.newpoint(name=name, coords=[(lon, lat)])
    pnt.description = status

output_path = "/tmp/wyoming_ghost_towns.kml"
kml.save(output_path)
print(f"KML saved to: {output_path}")

FileLink(output_path)


Found 23 ghost towns.
KML saved to: /tmp/wyoming_ghost_towns.kml


/tmp/wyoming_ghost_towns.kml